# Our first nipype workflow

**Learning objectives of this notebook: understanding how to link nodes together to write a simple workflow.**
***

You can think of a nipype workflow (or any neuroimaging pipeline) as a [Directed Acyclic Graph (or DAG)](https://en.wikipedia.org/wiki/Directed_acyclic_graph), or a series of operations that can be performed serially or in parallel, and in which loops are not allowed (that is, the output of a child cannot be the input of a parent). Think of water flowing down a mountain slope (your input data), splitting into smaller rivers (your processes), and ending into a lake (the output data).

In Nipype, each vertex in the graph is either a `Node` object, which wraps an `Interface`, or another `Workflow`. Note that by using other workflows as vertices, nipype can generate modular pipelines, which will be useful when developing more complicated pipelines. In addition, it makes testing the individual parts of your pipeline much easier.

Writing nipype workflows is fairly straightforward after you have enough practice with the syntax. In my experience, the best way to write a workflow is by first drawing it on paper, and then moving to the code. Let's write a very simple pipeline that does the following on the input data

1. perform motion correction
2. estimate TSNR on the motion-corrected data
3. take the motion correction parameters and estimate framewise displacement

Or as a graph:

In [ ]:
import networkx as nx
from nxpd import draw
G = nx.DiGraph()
G.graph['dpi'] = 120
G.add_nodes_from(['Input', 'Motion Correction', 'TSNR', 'Framewise Displacement', 'Output'])
G.add_edges_from([
    ('Input', 'Motion Correction'),
    ('Motion Correction', 'TSNR'),
    ('Motion Correction', 'Framewise Displacement'),
    ('TSNR', 'Output'),
    ('Motion Correction', 'Output'),
    ('Framewise Displacement', 'Output')
])
draw(G, show='ipynb')

In [ ]:
from nipype import Workflow, Node, IdentityInterface
from nipype.interfaces import afni
from nipype.interfaces import fsl
from nipype.algorithms.confounds import FramewiseDisplacement, TSNR

First, we need to setup an input node and an output node. For this special type of node we are going to use an `IdentityInterface`. This interface simply creates identity mappings. They are used to collect input and output files.

In [ ]:
inputnode = Node(
    IdentityInterface(
        # fields specify what input/output fields this node will have.
        # In this case, the node `inputnode` will have an input called `in_file` 
        # and an output called `in_file`
        fields=['in_file']),
    name='inputnode'
)

outputnode = Node(
    IdentityInterface(fields=['tsnr_file', 'moco_file', 'fd']),
    name='outputnode'
)

Then we can create the intermediate nodes that will perform actions on the input data. We will use FSL's `MCFLIRT` for motion correction, AFNI's `3dTstat` for computing tSNR, and Nipype's `algorithms.confounds.FramewiseDisplacement` to compute framewise displacement from MCFLIRT's motion parameters. This example also shows you how easy it is to mix and match tools from different neuroimaging toolkits.

In [ ]:
flirt = Node(
    fsl.MCFLIRT(output_type='NIFTI_GZ', save_plots=True), 
    name='flirt'
)
fd = Node(
    FramewiseDisplacement(parameter_source='FSL'),
    name='compute_fd'
)

# we are going to use afni to compute tSNR
# the option -cvarinv of 3dTstat computes tSNR after linear detrending
tsnr = Node(
    afni.TStat(outputtype='NIFTI_GZ', args='-cvarinv'),
    name='compute_tsnr'
)

Now we are ready to link all these nodes together. First, we create an instance of a `Workflow` object. The only mandatory argument is its name.

In [ ]:
wf = Workflow(name='my_first_workflow')

Then, we are ready to connect the nodes using the method `Workflow.connect`. This method can be used in two ways. In its simplest form, the syntax is

```python
Workflow.connect(in_node, "output_field", out_node, "input_field")
```

that is, we are linking `in_node` to `out_node` by passing `"output_field"` (of `in_node`) to `"input_field"` (of `out_node`).

In [ ]:
# first way to connect nodes
# wf.connect(in_node, output_field, out_node, input_field)
#
# input data -> flirt
wf.connect(inputnode, 'in_file', flirt, 'in_file')

Alternatively, if we need to link many fields of the same node to another node, we can use the syntax

```python
Workflow.connect([  # this is a list of tuple
    # each tuple has three elements
    # the first two elements are the input and output nodes
    (in_node, out_node, # <-- which nodes to connect?
       # the last element is a list of tuples indicating the fields to connect
       [("output_field_1", "input_field_1"),  # <-- which fields to connect?
        ("output_field_2", "input_field_2")])
])
```

This is very confusing at first, and you might get lost in brackets (use an IDE for this). However, when you get used to it, it becomes very efficient and you will never use the first syntax.

In [ ]:
wf.connect([
    # moco params -> framewise displacement
    (flirt, fd, [('par_file', 'in_file')]),
    # moco file -> tsnr
    (flirt, tsnr, [('out_file', 'in_file')]),
    # moco file -> output node
    (flirt, outputnode, [('out_file', 'moco_file')]),
    # tsnr file -> output node
    (tsnr, outputnode, [('out_file', 'tsnr_file')]),
    # framewise displacement -> output node
    (fd, outputnode, [('out_file', 'fd')])
])

That's it. We created our first workflow! We can visualize it to make sure we have connected everything properly.

In [ ]:
wf.write_graph('myfirstworkflow')

from IPython.display import Image
Image('myfirstworkflow.png')

It looks OK! Let's now get some data and see if it works.

## Download some data from OpenNeuro

We are going to download some data from OpenNeuro using [DataLad](https://www.datalad.org/). This is not necessary for Nipype, but I hope these three lines of code will pique your curiosity about the wonderful possibilities of DataLad ;-)

In [ ]:
# Need the following to fix asyncio issues with jupyter
# see https://github.com/datalad/datalad/issues/5409
import nest_asyncio
nest_asyncio.apply()

# import datalad to download the data
import datalad.api as dl

# Download one of my datasets from OpenNeuro ("The Grand Budapest Hotel")
# https://openneuro.org/datasets/ds003017
# We will be using datalad for this
dl.install('budapest-data', '///openneuro/ds003017')

Let's check what we have downloaded

In [ ]:
!ls budapest-data/

In [ ]:
!tree budapest-data/sub-sid000021 | head -20

For now we have only downloaded a list of files, without the actual file content (the content is saved to git with [git-annex](https://git-annex.branchable.com)). To get the data, we need to tell datalad to download the file we want.

In [ ]:
# Just get one subject's run.
data_fn = 'budapest-data/sub-sid000021/func/sub-sid000021_task-movie_run-01_bold.nii.gz'
dl.get(data_fn)

Let's check if we have a brain in there.

In [ ]:
import nibabel as nib
import cortex
import matplotlib.pyplot as plt

data = nib.load(data_fn).get_fdata()
print(f"Shape of data is {data.shape}")
# compute mean across TRs
data_avg = data.mean(-1)
# plot mosaic
plt.figure(figsize=(12, 12))
_ = cortex.mosaic(data_avg[..., 10:40].T, cmap='gray', vmin=0, vmax=500)

GOOD, it looks like a brain! We are going to select the first 50 TRs to make our example run faster.

In [ ]:
from nilearn.image import index_img
import os

# do not use relative paths
data_fn = os.path.abspath(data_fn)
# get the first 50 TRs
data_subset = index_img(data_fn, slice(0, 50))

print(f"Shape of data_subset is {data_subset.shape}")
# save to test_data
test_data_fn = os.path.abspath('test_data.nii.gz')
data_subset.to_filename(test_data_fn)

print(f"Test data is in {test_data_fn}")

## Let's run our workflow

By default our workflow will run in the current directory. This is not great. We can set a work directory where all the intermediate files will be stored. This directory will be used for caching.

In [ ]:
# set up a "work directory"
wf.base_dir = os.path.abspath('./workdir')

Now we are ready to pass our inputfile to the workflow. We do so by setting the `in_file` field of the `inputnode`.

In [ ]:
# Note that we are selecting the inputs first (wf.inputs),
# then we are specifying which node we want to modify (wf.inputs.inputnode),
# and finally we set the parameter in_file 
wf.inputs.inputnode.in_file = test_data_fn

And now we are ready to run our workflow!

In [ ]:
# run it -- by default it's going to run sequentially with one thread
wf.run()

Now all our results are under the working directory

In [ ]:
!tree workdir/my_first_workflow/

Each node (except the `inputnode` and the `outputnode`) has an associated folder containing the node outputs. For now we are going to ignore the other files. However, note that nodes wrapping command line arguments (such as `flirt`) contain a file named `command.txt`. This file contains the command that was run to obtain the output file. For debugging, it's often useful to check this command and run it in a terminal.

In [ ]:
!cat workdir/my_first_workflow/flirt/command.txt

Let's check the output tSNR volume and the framewise displacement

In [ ]:
tsnr_img = nib.load('workdir/my_first_workflow/compute_tsnr/test_data_mcf_tstat.nii.gz')
tsnr_data = tsnr_img.get_fdata()

plt.figure(figsize=(12, 12))
_ = cortex.mosaic(tsnr_data.T, cmap='plasma', vmin=0, vmax=130)

In [ ]:
import numpy as np
fd_data = np.loadtxt('workdir/my_first_workflow/compute_fd/fd_power_2012.txt', skiprows=1)

fig, ax = plt.subplots(1, 1, figsize=(12, 2))
ax.plot(fd_data)
ax.set_ylim([0, .4])
ax.set_ylabel('FD [mm]')
ax.set_xlabel('TR')
ax.grid(axis='y')
_ = ax.set_title('Framewise Displacement')

*** 
In this notebook we created our first workflow. However, we are not done yet. We often want to apply the same workflow to multiple input files. And we do not need to dig files out of the work directory every time we run our pipelines. In the next notebook, we will extend our simple pipeline to run it over multiple input files and we will generate a nicely-formatted output directory for all our output files.

Go to the next notebook: [03-extending-our-first-workflow](03-extending-our-first-workflow.ipynb).